https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

### Single layer nerual network 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [54]:
df = pd.read_csv('seeds_dataset.csv', header = None)
df.head()

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [55]:
print (df.columns)
df.iloc[:, 7].unique()

Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')


array([1, 2, 3], dtype=int64)

In [61]:
# convert column names from integer to string 

df.columns = df.columns.map(str)
print (df.columns)

Index(['0', '1', '2', '3', '4', '5', '6', '7'], dtype='object')


In [59]:
df['7'].unique()

array([1, 2, 3], dtype=int64)

### initialzied the network

In [8]:
# n_hidden: number of neurons in each hidden layer and each each hidden layer has n_input + 1 weights 
# n_outputs: number of neurons in the output layer and output layer has n_hidden + 1 weights 


def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    # weight matrix {num_row, num_col}
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    
    return network

In [9]:
from random import seed
from random import random

seed(1)
# test initialize_network - two dimensional input data, one hidden layer with one neuron, binary output data 
network = initialize_network(2, 1, 2)

for layer in network:
    print (layer)
    
print ('==')

for layer in network:
    for neuron in layer:
        print (neuron)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]
==
{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}
{'weights': [0.2550690257394217, 0.49543508709194095]}
{'weights': [0.4494910647887381, 0.651592972722763]}


### forward propagation 

In [10]:
# linear weight function -- sum(weight_i * input_i) + bias

def activate(weights, inputs):
    # assume bias is the last entry in the weight vector
    activation = weights[-1]
    
    for i in range(len(weights) - 1):
        # inputs are row data (with each data point from each attribute) in the training data - elementwise multiplicaiotn
        activation += weights[i] * inputs[i]
        
    return activation 

In [11]:
# use sigmoid function as activation function to transfer the linear weight function to a range between(0, 1)
# g(z) = 1/(1 + exp(-z)) where z is the linear weight function defined in the above cell as activate(-, -)

from math import exp

def transfer (activation):
    return 1/(1 + exp(-activation))

In [64]:
# forward propagate each row in the dataset through the netwrok and 
# save the forward propagate result from each neuron as 'output' after the neuron's weight vector and
# return those activated weighted inputs through the entire netwrok, which is also the model prediction for a given row of data

def forward_propagate(network, row):
    inputs = row
    
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])  
        # to use out/activated weighted inputs from the previous layer as input for the next layer 
        inputs = new_inputs
#         print(inputs)
        
    return inputs

In [68]:
# forward propagation calculation 
# print (transfer(0.13436424411240122 + 0.763774618976614 ))
# print (transfer(0.7105668883115941 * 0.2550690257394217 + 0.49543508709194095 ))

0.7105668883115941
0.6629970129852887


In [71]:
# test forward propagate

# one example row of data
row = [1, 0, None]

seed(1)
network = initialize_network(2, 1, 2)
print(network)
print ('--')
output = forward_propagate(network, row)
print ('--')
print (output)


[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}], [{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
--
[0.7105668883115941]
[0.6629970129852887, 0.7253160725279748]
--
[0.6629970129852887, 0.7253160725279748]


### back propagation

http://www.bogotobogo.com/python/scikit-learn/Artificial-Neural-Network-ANN-4-Backpropagation.php

In [14]:
# derivative of sigmoid function folows dg(z)/ d(z) = g(z) (1 - g(z)) where z is the weighted input function

# derivative of one neuron  
def transfer_derivative(output):
    return output * (1 - output)

In [122]:
# from the link above, to get the gradient for weight update -
# output layer error: delta3 = (y - yhat) * transfer_derivative(forward_propagate- output layer)
# hidden layer error: delta2 = weight_jk * delta3 * transfer_derivative(forward_propagate - hidden layer)
# error_k: error from the kth neuron in the output layer 
# weight_jk: weight that connects the jth neuron in the previous layer to the current neuron k, which is the kth neuron in the output layer 

def backward_propagate_error(network, target):
    
    # start from the last layer 
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        
        if i == len(network) - 1: 
            for j in range(len(layer)): 
                # for each neuron in the output layer, get the list of errors (y - yhat) from all the neurons in the output layer
                neuron = layer[j]
                errors.append(target[j] - neuron['output'])
                                
        else:
            for j in range(len(layer)):
                error = 0
                # delta2 = error of each neuron in the output layer/delta3 weighted by the output layer weight matrix/ W_out
                for neuron in network[i + 1]:
#                     print (('delta3 {} ').format(neuron['delta']))
                    
#                     print (('W2 {} ').format(neuron['weights'][j]))
                    error += neuron['weights'][j] * neuron['delta']
                
                errors.append(error)
                
        # first goes through ouput layer then hidden layer to update delta (changes in error)
        for j in range(len(layer)):
            # first run occurs when it's the output layer weight gradient - delta3 = (y - yhat) * transfer_derivative(value from forward_propagate at the corresponding layer/neuron)
            # second run occurs when it's the output layer weight gradient - delta2 = W_out * delta3 * transfer_derivative(value from forward_propagate at the corresponding layer/neuron)
            
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
            
#             print (neuron['delta'])

In [121]:
# test back propagation

target = [0, 1]
backward_propagate_error(network, target)

print('--')
for layer in network:
    for neuron in layer:
        print(neuron)

delta3 -0.09563551015402987 
W2 -0.018621568544238264 
delta3 0.09442372836861457 
W2 0.14004633794102356 
delta3 -0.09563551015402987 
W2 -0.578388013685489 
delta3 0.09442372836861457 
W2 0.708209999915483 
--
{'weights': [0.6258665632560454, 0.5421720203506891, 0.23659694663756192], 'output': 0.9990340005457791, 'delta': 1.4480415052866149e-05}
{'weights': [0.41498988261693415, -0.3293962323767823, 0.6157968062551293], 'output': 0.9093210680525435, 'delta': 0.010075022536625235}
{'weights': [-0.018621568544238264, -0.578388013685489, -0.0007479773687590341], 'output': 0.3988617743466267, 'delta': -0.09563551015402987}
{'weights': [0.14004633794102356, 0.708209999915483, -0.2245600506531873], 'output': 0.6049111444282086, 'delta': 0.09442372836861457}


In [115]:
for layer in network:
    for neuron in layer:
        print(neuron['output'])

print ('--')

# delta3 calculation double checking 
print ((0 - 0.6629970129852887) * transfer_derivative(0.6629970129852887))
print ((1 - 0.7253160725279748) * transfer_derivative(0.7253160725279748))

print ('--')

# delta2 calculation double checking 
a = -0.13436424411240122 * 0.14813473120687762 
b = 0.8474337369372327 * 0.05472601157879688
c = transfer_derivative(0.7253160725279748)

print ((a + b) * c)

0.7105668883115941
0.6629970129852887
0.7253160725279748
--
-0.14813473120687762
0.05472601157879688
--
0.005274218131534226


### train network

In [123]:
# update network 'weights' with error using Stochastic Gradient Descent: weight = weight + learning rate * error * input

def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:, -1]
    
        if i != 0:
            inputs = [neuron['output'] for neuron in network [i - 1]]
        
        for neuron in network[i]:
            for j in range(len(inputs)):
                # input here is the output in each neuron from forward_propagation function
                # dL/dW2
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            # bias 
            neuron['weights'][-1] += l_rate * neuron['delta']
            

$$
\begin{aligned}
\delta_3 = (y - \hat{y})  df(z_3)\\
\frac{\partial{L}}{\partial{W_2}} = a_2^T \delta_3  \\
\frac{\partial{L}}{\partial{W_1}} = x^T \delta_2\\
\delta_2 = \delta_3 W_2^ Tdf(z_2) \\
\frac{\partial{L}}{\partial{b_2}} = \delta_3\\
\frac{\partial{L}}{\partial{b_1}} = \delta_2 \\
\end{aligned}
$$

In [124]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
        
		for row in train:
            # forward_propagate function output is created based on the number of neurons in the output layer s
			outputs = forward_propagate(network, row)
            
            # output label vector size 
			target = [0 for i in range(n_outputs)]
            # create one hot encoding label(class 1: [0, 1], class 2:[1, 0]) 
			target[row[-1]] = 1
            
			sum_error += sum([(target[i] - outputs[i]) ** 2 for i in range(len(target))])
            
			backward_propagate_error(network, target)
			update_weights(network, row, l_rate)
            
		print('>epoch = %d, lrate = %.3f, error = %.3f' % (epoch, l_rate, sum_error)) 

In [132]:
# test training 
# synthetic small dataset randoms

dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

# for mdoel parameter initialization
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))

network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)

for layer in network:
    print(layer)

[{'weights': [0.7193680254752827, 0.645928898519349, 0.0766473024361174], 'output': 0.9996149909665241, 'delta': 3.673037513715458e-05}, {'weights': [-1.1650269234293638, 1.4194315320972306, 0.44119923670919975], 'output': 0.03887528463040981, 'delta': -0.008616012348643742}]
[{'weights': [-0.5793106825227193, 2.0767938588890926, -0.4660219839317919], 'output': 0.2879608270767656, 'delta': -0.0590433120818058}, {'weights': [0.8595711203410333, -1.4056934626472715, -0.06976859326112657], 'output': 0.6588363075867731, 'delta': 0.07668371365263751}]


### prediction

In [37]:
# make prediction with the network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [38]:
# on the same small dataset used before

for row in dataset:
	prediction = predict(network, row)
	print('Target = %d, Got = %d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
